In [27]:
# importing required libraries.
import pandas as pd
import numpy as np
import csv
# uploading train and test data.
df = pd.read_csv("F:/ai2000/assign/a4/new-york-city-taxi-fare-prediction/train.csv",delimiter=',', nrows = 200000)
df1 = pd.read_csv("F:/ai2000/assign/a4/new-york-city-taxi-fare-prediction/test.csv",delimiter=',')
## Analyzing data for cleansing.
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,200000.000000,200000.000000,200000.000000,199999.000000,199999.000000,200000.000000
mean,11.342877,-72.506121,39.922326,-72.518673,39.925579,1.682445
std,9.837855,11.608097,10.048947,10.724226,6.751120,1.306730
min,-44.900000,-736.550000,-3116.285383,-1251.195890,-1189.615440,0.000000
25%,6.000000,-73.992050,40.735007,-73.991295,40.734092,1.000000
50%,8.500000,-73.981743,40.752761,-73.980072,40.753225,1.000000
75%,12.500000,-73.967068,40.767127,-73.963508,40.768070,2.000000
max,500.000000,2140.601160,1703.092772,40.851027,404.616667,6.000000


In [28]:
submission = pd.DataFrame()
submission['key'] = df1['key']

In [29]:
df1.describe()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,-73.974722,40.751041,-73.973657,40.751743,1.671273
std,0.042774,0.033541,0.039072,0.035435,1.278747
min,-74.252193,40.573143,-74.263242,40.568973,1.000000
25%,-73.992501,40.736125,-73.991247,40.735254,1.000000
50%,-73.982326,40.753051,-73.980015,40.754065,1.000000
75%,-73.968013,40.767113,-73.964059,40.768757,2.000000
max,-72.986532,41.709555,-72.990963,41.696683,6.000000


In [30]:
# As we can see that some fare_amounts are negative. So we should remove it.
df = df[df.fare_amount>=0]
# Checking for missing values in the data.
print(df.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64


In [31]:
# As we can see that there are some missing values present in the data. So removing those rows.
df = df.dropna()
print(df.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


In [32]:
# Checking fare_amount column.
df['fare_amount'].describe()

count    199986.000000
mean         11.344032
std           9.836736
min           0.000000
25%           6.000000
50%           8.500000
75%          12.500000
max         500.000000
Name: fare_amount, dtype: float64

In [33]:
# Checking passenger_count column.
df['passenger_count'].describe()

count    199986.000000
mean          1.682423
std           1.306692
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           6.000000
Name: passenger_count, dtype: float64

In [34]:
# Dropping a row if passenger_count>6 as a taxi cannot carry more than 6 passengers at a time.
df = df.drop(df[df['passenger_count']>6].index, axis = 0)
df['passenger_count'].describe()

count    199986.000000
mean          1.682423
std           1.306692
min           0.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           6.000000
Name: passenger_count, dtype: float64

In [35]:
# Now let us observe pickup and drop off coordinates.
## In general latitudes should lie between -90 to +90 degree
## In general longitudes should lie between -180 to +180 degree
### So, we need to drop the outliers
print("pickup_latitude:\n",df['pickup_latitude'].describe(),"\n")
print("pickup_longitude:\n",df['pickup_longitude'].describe(),"\n")
print("dropoff_latitude:\n",df['dropoff_latitude'].describe(),"\n")
print("dropoff_longitude:\n",df['dropoff_longitude'].describe())

pickup_latitude:
 count    199986.000000
mean         39.922268
std          10.049296
min       -3116.285383
25%          40.735006
50%          40.752761
75%          40.767126
max        1703.092772
Name: pickup_latitude, dtype: float64 

pickup_longitude:
 count    199986.000000
mean        -72.506021
std          11.608497
min        -736.550000
25%         -73.992050
50%         -73.981743
75%         -73.967070
max        2140.601160
Name: pickup_longitude, dtype: float64 

dropoff_latitude:
 count    199986.000000
mean         39.925729
std           6.750746
min       -1189.615440
25%          40.734092
50%          40.753225
75%          40.768070
max         404.616667
Name: dropoff_latitude, dtype: float64 

dropoff_longitude:
 count    199986.000000
mean        -72.518951
std          10.723343
min       -1251.195890
25%         -73.991296
50%         -73.980072
75%         -73.963509
max          40.851027
Name: dropoff_longitude, dtype: float64


In [36]:
# We are observing that there are some outliers present in the data.
## Dropping those rows from the data.
df = df.drop(((df[df['pickup_latitude']<-90])).index, axis=0)
df = df.drop(((df[df['pickup_latitude']>90])).index, axis=0)
df = df.drop(((df[df['pickup_longitude']<-180])).index, axis=0)
df = df.drop(((df[df['pickup_longitude']>180])).index, axis=0)
df = df.drop(((df[df['dropoff_latitude']<-90])).index, axis=0)
df = df.drop(((df[df['dropoff_latitude']>90])).index, axis=0)
df = df.drop(((df[df['dropoff_longitude']<-180])).index, axis=0)
df = df.drop(((df[df['dropoff_longitude']>180])).index, axis=0)
print("pickup_latitude:\n",df['pickup_latitude'].describe(),"\n")
print("pickup_longitude:\n",df['pickup_longitude'].describe(),"\n")
print("dropoff_latitude:\n",df['dropoff_latitude'].describe(),"\n")
print("dropoff_longitude:\n",df['dropoff_longitude'].describe())

pickup_latitude:
 count    199980.000000
mean         39.927916
std           6.057515
min         -74.007670
25%          40.735008
50%          40.752762
75%          40.767125
max          69.400000
Name: pickup_latitude, dtype: float64 

pickup_longitude:
 count    199980.000000
mean        -72.514107
std          10.394136
min         -78.083332
25%         -73.992050
50%         -73.981743
75%         -73.967071
max          40.811147
Name: pickup_longitude, dtype: float64 

dropoff_latitude:
 count    199980.000000
mean         39.928220
std           6.056889
min         -74.035197
25%          40.734092
50%          40.753225
75%          40.768070
max          43.415190
Name: dropoff_latitude, dtype: float64 

dropoff_longitude:
 count    199980.000000
mean        -72.513021
std          10.394533
min         -84.654241
25%         -73.991295
50%         -73.980072
75%         -73.963509
max          40.851027
Name: dropoff_longitude, dtype: float64


In [12]:
#print("pickup_latitude:\n",df1['pickup_latitude'].describe(),"\n")
#print("pickup_longitude:\n",df1['pickup_longitude'].describe(),"\n")
#print("dropoff_latitude:\n",df1['dropoff_latitude'].describe(),"\n")
#print("dropoff_longitude:\n",df1['dropoff_longitude'].describe())

In [13]:
# Observing data types and changing the object data types into accessible datetime type.

In [37]:
# Observing data types of both training and testing data.
print("Training data:\n",df.dtypes,"\n")
print("Test data:\n",df1.dtypes,"\n")
df['key'] = pd.to_datetime(df['key'])
df['pickup_datetime']  = pd.to_datetime(df['pickup_datetime'])
df1['key'] = pd.to_datetime(df1['key'])
df1['pickup_datetime']  = pd.to_datetime(df1['pickup_datetime'])

Training data:
 key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object 

Test data:
 key                   object
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object 



In [38]:
print("Training data after changing:\n",df.dtypes,"\n")
print("Test data after changing:\n",df1.dtypes,"\n")

Training data after changing:
 key                       datetime64[ns]
fare_amount                      float64
pickup_datetime      datetime64[ns, UTC]
pickup_longitude                 float64
pickup_latitude                  float64
dropoff_longitude                float64
dropoff_latitude                 float64
passenger_count                    int64
dtype: object 

Test data after changing:
 key                       datetime64[ns]
pickup_datetime      datetime64[ns, UTC]
pickup_longitude                 float64
pickup_latitude                  float64
dropoff_longitude                float64
dropoff_latitude                 float64
passenger_count                    int64
dtype: object 



In [39]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.000000100,4.5,2009-06-15 17:26:21+00:00,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.000000200,16.9,2010-01-05 16:52:16+00:00,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.000000490,5.7,2011-08-18 00:35:00+00:00,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.000000100,7.7,2012-04-21 04:30:42+00:00,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968095,40.768008,-73.956655,40.783762,1


In [40]:
data = [df]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day of Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

data1 = [df1]
for i in data1:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day of Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

In [41]:
#pickup latitude and longitude = 0
df = df.drop(df.loc[((df['pickup_latitude']==0) & (df['pickup_longitude']==0))&((df['dropoff_latitude']!=0) & (df['dropoff_longitude']!=0)) & (df['fare_amount']==0)].index,axis=1)

In [42]:
#dropoff latitude and longitude = 0
df = df.drop(df.loc[((df['pickup_latitude']!=0) & (df['pickup_longitude']!=0))&((df['dropoff_latitude']==0) & (df['dropoff_longitude']==0)) & (df['fare_amount']==0)].index,axis=1)

In [43]:
df = df.drop(['key','pickup_datetime'], axis = 1)
df1 = df1.drop(['key','pickup_datetime'], axis = 1)
X_train = df.iloc[:,df.columns!='fare_amount']
y_train = df['fare_amount'].values
X_test = df1

In [44]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [45]:
submission['fare_amount'] = y_pred
submission.to_csv('sub0.csv', index=False)

In [46]:
# build the lightgbm model
import lightgbm as lgb
model = lgb.LGBMRegressor(num_leaves=31,learning_rate=0.05,max_depth= -1,subsample= 0.8,subsample_freq=1,)
model.fit(X_train, y_train)
# predicting on test data
y_pred = model.predict(X_test)
print(y_pred)
submission['fare_amount'] = y_pred
submission.to_csv('sub1.csv', index=False)

[ 9.05362718  9.61645658  7.39193543 ... 53.87428818 20.7764783
  8.70201577]


In [47]:
# XGBRegressor
from xgboost import XGBRegressor

# fit model on training data
model = XGBRegressor(n_estimators=200, learning_rate=0.23, max_depth=5, random_state=0)
model.fit(X_train, y_train)
# predicting on test data
y_pred = model.predict(X_test)
print(y_pred)
submission['fare_amount'] = y_pred
submission.to_csv('sub2.csv', index=False)

[ 9.103869   9.040025   5.7787156 ... 50.917194  20.164333   7.0950446]
